In [ ]:
# # -*- coding: utf-8 -*-
# import torch
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     TrainingArguments,
#     Trainer,
#     BitsAndBytesConfig
# )
# from peft import LoraConfig, get_peft_model
# from datasets import Dataset
# import json
# from docx import Document
# from rouge_score import rouge_scorer
# import os

# # =============================================
# # Step 1: Document Processing and Dataset Creation
# # =============================================

# def docx_to_text(docx_path):
#     """Extract text from docx files"""
#     doc = Document(docx_path)
#     return "\n".join([para.text for para in doc.paragraphs if para.text.strip()])


# def create_qa_pairs(text, window_size=512, stride=256):
#     """Create synthetic QA pairs from document text"""
#     qa_pairs = []
#     words = text.split()
    
#     for i in range(0, len(words), stride):
#         chunk = " ".join(words[i:i+window_size])
#         # Create simple questions (you should customize this for your documents)
#         qa_pairs.append({
#             "question": f"Summarize the following text: {chunk[:100]}...",
#             "context": chunk,
#             "answer": chunk[:512]  # Simple demonstration answer
#         })
#     return qa_pairs


# def prepare_dataset(docx_folder, output_json="dataset.json"):
#     """Process folder of docx files into QA dataset"""
#     dataset = []
    
#     for file in os.listdir(docx_folder):
#         if file.endswith(".docx"):
#             text = docx_to_text(os.path.join(docx_folder, file))
#             dataset += create_qa_pairs(text)
    
#     with open(output_json, "w") as f:
#         json.dump(dataset, f)
        
#     return Dataset.from_json(output_json)

# # =============================================
# # Step 2: Model Configuration (FIXED)
# # =============================================

# model_name = "Qwen/Qwen-1_8B-Chat"
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

# # Initialize tokenizer with pad_token set explicitly
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     pad_token='<|endoftext|>',  # Set pad_token during initialization
#     padding_side="left"
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=quant_config,
#     device_map="auto",
#     trust_remote_code=True,
#     use_flash_attn=False
# )

# # =============================================
# # Step 3: LoRA Configuration for Efficient Tuning
# # =============================================

# peft_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     target_modules=["c_attn", "c_proj", "w1", "w2"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

# # =============================================
# # Step 4: Data Processing
# # =============================================

# def format_instruction(sample):
#     return f"""You are a document assistant. Answer based on the context.
# Context: {sample['context']}
# Question: {sample['question']}
# Answer: {sample['answer']}"""

# def preprocess_dataset(dataset):
#     def tokenize_func(examples):
#         # Process batch entries by zipping corresponding elements
#         texts = [
#             format_instruction({
#                 "question": q,
#                 "context": c,
#                 "answer": a
#             }) for q, c, a in zip(
#                 examples['question'],
#                 examples['context'],
#                 examples['answer']
#             )
#         ]
#         return tokenizer(
#             texts,
#             max_length=512,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt" 
#         )
    
#     return dataset.map(tokenize_func, batched=True)


# # =============================================
# # Step 5: Training Configuration
# # =============================================

# training_args = TrainingArguments(
#     output_dir="./qwen-finetuned",
#     num_train_epochs=3,
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=4,
#     learning_rate=2e-5,
#     fp16=True,
#     logging_steps=10,
#     save_strategy="epoch",
#     evaluation_strategy="no"
# )

# # =============================================
# # Step 6: Training Execution
# # =============================================

# # First prepare your dataset (put your docx files in './documents')
# dataset = prepare_dataset("/home/dusoudeth/Documentos/github/crew-scientific-research/research/context/docx")
# processed_dataset = preprocess_dataset(dataset).train_test_split(test_size=0.1)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=processed_dataset["train"],
# )

In [ ]:
# trainer.train()
# model.save_pretrained("./qwen-finetuned-lora")

In [ ]:
# # =============================================
# # Step 7: Evaluation
# # =============================================

# scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# def evaluate(model, tokenizer, dataset, num_samples=10):
#     model.eval()
#     scores = []
    
#     for example in dataset.shuffle().select(range(num_samples)):
#         prompt = format_instruction(example).split("Answer:")[0] + "Answer:"
#         inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=100,
#                 temperature=0.7
#             )
        
#         prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         reference = example["answer"]
        
#         scores.append(scorer.score(reference, prediction.split("Answer:")[-1].strip()))
    
#     return sum([s['rougeL'].fmeasure for s in scores])/len(scores)

# print(f"Average ROUGE-L Score: {evaluate(model, tokenizer, processed_dataset['test'])}")

In [ ]:
# # =============================================
# # Step 8: Inference Usage
# # =============================================

# def ask_question(question, context):
#     prompt = f"""You are a document expert. Answer based on the context.
# Context: {context}
# Question: {question}
# Answer:"""
    
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=200,
#         temperature=0.7,
#         top_p=0.9
#     )
    
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

- - -

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import json
from docx import Document
from rouge_score import rouge_scorer
import os

# =============================================
# Step 1: Document Processing and Dataset Creation
# =============================================

def docx_to_text(docx_path):
    """Extract text from docx files"""
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

def create_qa_pairs(text, window_size=512, stride=256):
    """Create synthetic QA pairs from document text"""
    qa_pairs = []
    words = text.split()
    
    for i in range(0, len(words), stride):
        chunk = " ".join(words[i:i+window_size])
        # Create simple questions (you should customize this for your documents)
        qa_pairs.append({
            "question": f"Summarize the following text: {chunk[:100]}...",
            "context": chunk,
            "answer": chunk[:512]  # Simple demonstration answer
        })
    return qa_pairs

def prepare_dataset(docx_folder, output_json="dataset.json"):
    """Process folder of docx files into QA dataset"""
    dataset = []
    
    for file in os.listdir(docx_folder):
        if file.endswith(".docx"):
            text = docx_to_text(os.path.join(docx_folder, file))
            dataset += create_qa_pairs(text)
    
    with open(output_json, "w") as f:
        json.dump(dataset, f)
        
    return Dataset.from_json(output_json)

# =============================================
# Step 2: Model Configuration (UPDATED)
# =============================================

model_name = "microsoft/phi-2"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Initialize tokenizer with Phi-2 specific settings
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="right",
    use_fast=False
)

# Phi-2 specific token configuration
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16  # Changed from fp16 to torch_dtype
)

# Verify model config matches tokenizer
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

special_tokens = {"additional_special_tokens": ["<|im_start|>", "<|im_end|>"]}
tokenizer.add_special_tokens(special_tokens)

# =============================================
# Step 3: LoRA Configuration for Efficient Tuning
# =============================================

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["Wqkv", "out_proj", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# =============================================
# Step 4: Data Processing (UPDATED)
# =============================================

def preprocess_dataset(dataset):
    def tokenize_func(examples):
        texts = [
            f"""<|im_start|>system
You are a helpful document assistant.<|im_end|>
<|im_start|>user
Context: {context}
Question: {question}<|im_end|>
<|im_start|>assistant
{answer}<|im_end|>"""
            for question, context, answer in zip(
                examples['question'],
                examples['context'],
                examples['answer']
            )
        ]
        
        tokenized = tokenizer(
            texts,
            max_length=256,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        
        # Create labels by shifting input_ids
        tokenized["labels"] = tokenized["input_ids"].clone()
        return tokenized
    
    return dataset.map(tokenize_func, batched=True)

# =============================================
# Step 5: Training Configuration (UPDATED)
# =============================================

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)

training_args = TrainingArguments(
    output_dir="./phi2-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,  # Reduced learning rate
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",
    gradient_checkpointing=True,
    optim="adafactor",  # More memory-efficient
    remove_unused_columns=False  # Important for custom datasets
)


# =============================================
# Step 6: Training Execution
# =============================================

# First prepare your dataset (put your docx files in './documents')
dataset = prepare_dataset("/home/dusoudeth/Documentos/github/crew-scientific-research/research/context/docx")
processed_dataset = preprocess_dataset(dataset).train_test_split(test_size=0.1)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    data_collator=data_collator  # Add proper data collator
)

import torch
torch.cuda.empty_cache()

trainer.train()
model.save_pretrained("./phi2-finetuned-lora")

.